In [3]:
# def calculate_metrics(predictions):
#     # 计算真正例（TP）、假正例（FP）、真负例（TN）、假负例（FN）
#     TP = sum(predictions)  # 真正例数量等于预测为1且实际为1的样本数
#     FP = len(predictions) - TP  # 假正例数量等于预测为1且实际为0的样本数
#     TN = 0  # 实际不存在
#     FN = 0  # 实际不存在

#     # 计算准确率
#     accuracy = TP / len(predictions)

#     # 计算召回率
#     if TP + FN != 0:
#         recall = TP / (TP + FN)
#     else:
#         recall = 0

#     # 计算 F1 分数
#     if TP + FP != 0 and TP + FN != 0:
#         f1_score = 2 * TP / (2 * TP + FP + FN)
#     else:
#         f1_score = 0

#     return accuracy, recall, f1_score

# # 示例使用
# predictions = [1, 0, 1, 1, 0, 1, 0, 0, 1, 1]  # 示例预测结果
# accuracy, recall, f1_score = calculate_metrics(predictions)
# print("Accuracy:", accuracy)
# print("Recall:", recall)
# print("F1 Score:", f1_score)


In [4]:
# import numpy as np
# val = ['B-poiCollection', 'B-poiCollection', 'B-poiCollection', 'I-poiCollection', 'B-poiCollection', 'I-poiCollection', 'E-poiCollection', 'E-poiCollection', 'I-houseNumber', 'I-houseNumber', 'I-houseNumber', 'I-houseNumber', 'I-houseNumber']
# v = np.array(val)
# v = np.where(v == 'B-poiCollection', 1, 0)
# v

In [5]:
# arr = ['B-poi','I-poi','E-poi','B-poi','B-poi','B-poi','B-poi','E-poi','I-poi','I-poi','I-poi','E-poi','E-poi', 'S-poi']

有这样的一组标注分别是B-poi,I-poi,E-poi,S-poi, 分别是如下意思:
B-poi: 词的开始
I-poi: 词的中间, 一个词如果只有两个字, 那么I就不存在
E-poi: 词的结束
S-poi: 单独成词

现在含有以上标签的的一个数组, 类似['B-poi','I-poi','E-poi','B-poi','B-poi','B-poi','B-poi','E-poi','I-poi','I-poi','I-poi','E-poi','E-poi', 'S-poi']

请用python代码统计词的数量. 例如上面的例子中, 词的数量是10, 注意, 非法的标签也单独成词
分别是['B-poi','I-poi','E-poi'], ['B-poi'], ['B-poi'], ['B-poi'], ['B-poi','E-poi'], ['I-poi'], ['I-poi'], ['I-poi', 'E-poi'], ['E-poi'], ['S-poi']



In [6]:
# tags = ['B-poi','I-poi','E-poi','B-poi','B-poi','B-poi','B-poi','E-poi','I-poi','I-poi','I-poi','E-poi','E-poi', 'S-poi']
# stack = []
# predict_one_group = []
# for i, token in enumerate(tags):
#   if token == 'S-poi':
#     predict_one_group.append([(token, i)])
#     continue

#   if len(stack) == 0:
#     stack.append((token, i))
#     continue

#   last_token, _ = stack[-1]
#   if last_token == 'B-poi' and token == 'I-poi' or last_token == 'B-poi' and token == 'E-poi' or last_token == 'I-poi' and token == 'E-poi':
#     stack.append((token, i))
#   else:
#     predict_one_group.append(stack)
#     stack = [(token, i)]

# if len(stack) > 0:
#   predict_one_group.append(stack)

# print("词的列表是：", predict_one_group)

In [7]:
# tokens = ['B-poi','I-poi','E-poi','B-poi','B-poi','B-poi','B-poi','E-poi','S-city', 'I-poi','I-poi','I-poi','E-poi','E-poi', 'S-poi']
# predict_one_group = group_tokens(tokens, 'poi')
# print(predict_one_group)
# print(len(predict_one_group))

In [8]:
# raw = "上海市浦东新区张江镇汉东居委会"
# tokens = ['B-poi','I-poi','E-poi','B-poi','B-poi','B-poi','B-poi','E-poi','S-city', 'I-poi','I-poi','I-poi','E-poi','E-poi', 'S-poi']
# predict_one_group = group_tokens(tokens, 'poi')

# texts = trans_group_token_to_text(predict_one_group, raw)
# print(texts)

In [9]:


"""
将一组展开的token, 归并合理的组
"""
def group_tokens(tokens, category):
  stack = []
  group = []
  for i, token in enumerate(tokens):
    if token not in [f'B-{category}', f'I-{category}', f'E-{category}', f'S-{category}']:
      continue      

    if token == f'S-{category}':
      group.append([(token, i)])
      continue

    if len(stack) == 0:
      stack.append((token, i))
      continue

    last_token, _ = stack[-1]

    if last_token == f'B-{category}' and token == f'I-{category}' or last_token == f'B-{category}' and token == f'E-{category}' or last_token == f'I-{category}' and token == f'E-{category}':
      stack.append((token, i))
    else:
      group.append(stack)
      stack = [(token, i)]

  if len(stack) > 0:
    group.append(stack)

  return sorted(group, key=lambda x: x[0][1])

def batch_group_tokens(batch_tokens, category):
  batch_group = []
  for tokens in batch_tokens:
    batch_group.append(group_tokens(tokens, category))
  return batch_group


"""
统计的结果
predict_group: 预测的group
y_group: 真实的group
note: 所有的group只保留了特定的category
"""
def statistics_group(predict_group, y_group):

  if len(predict_group) == 0 or len(y_group) == 0:
    return [], [], [], [], 0, 0

  # 预测成功的
  correct = []

  # 编码错误的
  error_encode = []

  # 预测失败的
  error_result = []

  # 未知错误的
  error_unknow = []

  # 准确率
  precision = 0

  # 召回率
  recall = 0

  # 预测失败的
  for predict_one_group in predict_group:
    if predict_one_group in y_group:
      correct.append(predict_one_group)
      continue

    # 上面的条件排除了预测成功的, 剩下只有预测失败的和编码错误的

    # 只有一个长度, 代表肯定不是编码错误, 上面不排除是预测成功, 所以是预测错误
    if len(predict_one_group) == 1:
      if predict_one_group[0][0].startswith('S'):
        error_result.append(predict_one_group)
    else:
      """
      长度大于1的情况: 有几种可能:
      1. 预测错误: 如有交集, 也就是当前的group下标在y中有元素跟他下标有交集, 那么是预测错误
      2. 如果没有交集
      2.1 编码错误: 不能编码
      2.2 未知错误: 可以编码
      """
      # 长度
      # 暂时这个策略 v0.1.0
      # 长度大于1的情况, 如果当前的group下标在y中有元素跟他下标有交集, 那么是预测错误, 如果没有交集, 那么是编码错误
      # 有没有部分预测对的
      intersection = False
      intersection_index = -1
      for i, y_one_group in enumerate(y_group):
        if len(set([x[1] for x in predict_one_group]) & set([x[1] for x in y_one_group])) > 0:
          intersection = True
          intersection_index = i
          break

      if intersection:
        error_result.append({'predict': predict_one_group, 'y': y_group[intersection_index]})
      else:
        # 判断是否可以编码
        if predict_one_group[0][0].startswith('B') and predict_one_group[-1][0].startswith('E'):
          error_unknow.append(predict_one_group)
        else:
          error_encode.append(predict_one_group)

    
  # 计算准确率
  precision = len(correct) / len(predict_group)

  # 计算召回率
  recall = len(correct) / len(y_group)

  return correct, error_result, error_encode, error_unknow, precision, recall


def batch_statistics_group(batch_predict_group, batch_y_group):
  corrects = []
  error_results = []
  error_encodes = []
  error_unknows = []
  precisions = []
  recalls = []
  precision_mean = 0
  recall_mean = 0

  for predict_group, y_group in zip(batch_predict_group, batch_y_group):
    correct, error_result, error_encode, error_unknow, precision, recall = statistics_group(predict_group, y_group)
    corrects.append(correct)
    error_results.append(error_result)
    error_encodes.append(error_encode)
    error_unknows.append(error_unknow)
    precisions.append(precision)
    recalls.append(recall)
  
  if len(precisions) > 0:
    precision_mean = sum(precisions) / len(precisions)
  
  if len(recalls) > 0:
    recall_mean = sum(recalls) / len(recalls)

  return corrects, error_results, error_encodes, error_unknows, precisions, recalls, precision_mean, recall_mean

In [10]:

predict_tokens = ['B-poi','I-poi','E-poi','B-poi','B-poi','B-poi','B-poi','E-poi','S-city', 'I-poi','I-poi','I-poi','E-poi','E-poi', 'S-poi']
y_tokens = ['B-poi','I-poi','E-poi','S-poi','B-poi','I-poi','E-poi','E-poi','S-city', 'B-poi','I-poi','E-poi','E-poi','S-poi', 'S-poi']
predict_group = group_tokens(predict_tokens, 'poi')
y_group = group_tokens(y_tokens, 'poi')
correct, error_result, error_encode, error_unknow, precision, recall = statistics_group(predict_group, y_group)
print(f'correct: {correct}')
print(f'error_result: {error_result}')
print(f'error_encode: {error_encode}')
print(f'error_unknow: {error_unknow}')
print(f'precision: {precision}')
print(f'recall: {recall}')


correct: [[('B-poi', 0), ('I-poi', 1), ('E-poi', 2)], [('S-poi', 14)]]
error_result: [{'predict': [('B-poi', 6), ('E-poi', 7)], 'y': [('B-poi', 4), ('I-poi', 5), ('E-poi', 6)]}, {'predict': [('I-poi', 11), ('E-poi', 12)], 'y': [('B-poi', 9), ('I-poi', 10), ('E-poi', 11)]}]
error_encode: []
error_unknow: []
precision: 0.2
recall: 0.25


In [11]:
# 读取csv数据
import csv
y = []
with open('../input/eval_y.csv', mode='r', encoding='utf-8') as file:
  reader = csv.reader(file)
  for row in reader:
    y.append(row)

predict = []
with open('../input/eval_predict.csv', mode='r', encoding='utf-8') as file:
  reader = csv.reader(file)
  for row in reader:
    predict.append(row)

r = []
with open('../input/eval_token.csv', mode='r', encoding='utf-8') as file:
  reader = csv.reader(file)
  for row in reader:
    r.append(row)

In [12]:

unique_category = ['transitLine', 'floor', 'subpoi', 'road', 'otherinfo', 'poiCollection', 'poi', 'cls', 'sep', 'community', 'room', 'devZone', 'neighborhood', 'assist', 'country', 'subArea', 'province', 'aoi', 'township', 'building', 'district', 'redundant', '_PAD', 'station', 'crossing', 'cell', 'city', 'houseNumber', 'sideRoad', 'village', 'villageGroup']

In [13]:
import numpy as np
import os
from datetime import datetime

def trans_group(group, raw):
  start = group[0][1]
  end = group[-1][1]
  return ''.join(raw[start:end+1])

def eval(predict, y, r, unique_category, log=False, directory=None):
  assert len(predict) == len(y) == len(r)

  all_precision_mean = []
  all_recall_mean = []
  
  for category in unique_category:
    predict_group = batch_group_tokens(predict, category)
    y_group = batch_group_tokens(y, category)
    corrects, error_results, error_encodes, error_unknows, precisions, recalls, precision_mean, recall_mean = batch_statistics_group(predict_group, y_group)
    if log:
      os.makedirs(directory, exist_ok=True)
      
      # 拼接文件路径
      timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

      # 预测错误
      lines = []
      for errors, raw in zip(error_results, r):
        if len(errors) == 0:
          continue
        for e in errors:
          predict_text = trans_group(e['predict'], raw)
          y_text = trans_group(e['y'], raw)
          lines.append(f"{predict_text},{y_text},{category}")

      if len(lines) > 0:
        with open(os.path.join(directory, f"error-results-{category}-{timestamp}.csv"), mode='w', encoding='utf-8') as file:
          file.write('\n'.join(lines))



      # 编码错误
      lines = []
      for errors, raw in zip(error_encodes, r):
        if len(errors) == 0:
          continue
        for e in errors:
          encode_text = trans_group(e, raw)
          lines.append(f"{encode_text},{category}")

      if len(lines) > 0:
        with open(os.path.join(directory, f"error-encodes-{category}-{timestamp}.csv"), mode='w', encoding='utf-8') as file:
          file.write('\n'.join(lines))

  
      # 未知错误
      lines = []
      for errors, raw in zip(error_unknows, r):
        if len(errors) == 0:
          continue
        for e in errors:
          unknow_text = trans_group(e, raw)
          lines.append(f"{unknow_text},{category}")

      if len(lines) > 0:
        with open(os.path.join(directory, f"error-unknows-{category}-{timestamp}.csv"), mode='w', encoding='utf-8') as file:
          file.write('\n'.join(lines))

    if precision_mean > 0:
      all_precision_mean.append(precision_mean)
    if recall_mean > 0:
      all_recall_mean.append(recall_mean)

    print(f'precisions: {precision_mean}, recalls: {recall_mean}, category: {category}')

  if len(precisions) > 0:
    precision = sum(all_recall_mean) / len(all_recall_mean)
  
  if len(recalls) > 0:
    recall = sum(recalls) / len(recalls)

  f1_score = 2 * precision * recall / (precision + recall)

  return precision, recall, f1_score


In [15]:
precision, recall, f1_score = eval(predict, y, r, unique_category, True, './eval-result')
print(f'precision: {precision}, recall: {recall}, f1_score: {f1_score}')

precisions: 0.0, recalls: 0.0, category: transitLine
precisions: 0.0, recalls: 0.0, category: floor
precisions: 0.0, recalls: 0.0, category: subpoi
precisions: 0.060546875, recalls: 0.029003906249999996, category: road
precisions: 0.0, recalls: 0.0, category: otherinfo
precisions: 0.043598051300771365, recalls: 0.10200182025707646, category: poiCollection
precisions: 0.0, recalls: 0.0, category: poi
precisions: 0.0, recalls: 0.0, category: cls
precisions: 0.0, recalls: 0.0, category: sep
precisions: 0.0, recalls: 0.0, category: community
precisions: 0.0, recalls: 0.0, category: room
precisions: 0.0, recalls: 0.0, category: devZone
precisions: 0.0, recalls: 0.0, category: neighborhood
precisions: 0.0, recalls: 0.0, category: assist
precisions: 0.0, recalls: 0.0, category: country
precisions: 0.0, recalls: 0.0, category: subArea
precisions: 0.0, recalls: 0.0, category: province
precisions: 0.0, recalls: 0.0, category: aoi
precisions: 0.0, recalls: 0.0, category: township
precisions: 0.0,

In [2]:
from collections import defaultdict

# 假设你有两个字典：
dict1 = {'key1': ['value1', 'value2'], 'key2': ['value3']}
dict2 = {'key2': ['value4'], 'key3': ['value5']}

# 创建一个 defaultdict，其默认值为一个空列表
merged_dict = defaultdict(list)

# 将 dict1 和 dict2 的键值对添加到 merged_dict 中
for d in (dict1, dict2):  # 可以添加更多的字典
    for key, value in d.items():
        merged_dict[key].extend(value)

# 此时，merged_dict 的内容为：
# {'key1': ['value1', 'value2'], 'key2': ['value3', 'value4'], 'key3': ['value5']}
merged_dict

defaultdict(list,
            {'key1': ['value1', 'value2'],
             'key2': ['value3', 'value4'],
             'key3': ['value5']})